### Mapping table associating awards to works, based on backfill table `openalex.mid.work_funder`

In [0]:
%sql
CREATE OR REPLACE TABLE openalex.awards.work_awards
USING delta
AS
WITH work_funders as (
  SELECT
    paper_id,
    funder_id,
    funder_award_id
  FROM openalex.mid.work_funder
  LATERAL VIEW EXPLODE(award_ids) AS funder_award_id
),
awards as (
  SELECT
    id,
    display_name,
    funder_id,
    funder_award_id,
    funder
  FROM openalex.awards.openalex_awards
)
SELECT
  wf.paper_id as work_id,
  struct(
    concat("https://openalex.org/G", a.id) as id,
    a.display_name,
    a.funder_award_id,
    a.funder.id as funder_id,
    a.funder.display_name as funder_display_name
  ) as award
FROM
  work_funders wf
  JOIN
    awards a
  ON
    wf.funder_id = a.funder_id AND
    wf.funder_award_id = a.funder_award_id